In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('Classificacao com Spark')\
    .config("spark.driver.bindAddress", "localhost") \
    .config("spark.ui.port", "4050") \
    .config("spark.driver.memory", "4g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

In [4]:
spark

In [5]:
dataset = spark.read.parquet('Dataset Modelo/', inferSchema = True)

In [6]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Conjuge: integer (nullable = true)
 |-- Dependentes: integer (nullable = true)
 |-- TelefoneFixo: integer (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = true)
 |-- SegurancaOnline: integer (nullable = true)
 |-- BackupOnline: integer (nullable = true)
 |-- SeguroDispositivo: integer (nullable = true)
 |-- SuporteTecnico: integer (nullable = true)
 |-- TVaCabo: integer (nullable = true)
 |-- StreamingFilmes: integer (nullable = true)
 |-- ContaCorreio: integer (nullable = true)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContrato_DoisAnos

In [7]:
dataset.toPandas()

,id,Mais65anos,MesesDeContrato,MesesCobrados,Churn,Conjuge,Dependentes,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,...,Internet_DSL,Internet_FibraOptica,Internet_Nao,TipoContrato_Mensalmente,TipoContrato_UmAno,TipoContrato_DoisAnos,MetodoPagamento_DebitoEmConta,MetodoPagamento_CartaoCredito,MetodoPagamento_BoletoEletronico,MetodoPagamento_Boleto
0,7982,0,1,45.305408,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
1,9465,0,60,103.614223,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
2,2122,0,12,75.850000,0,0,0,1,1,0,...,1,0,0,0,1,0,0,0,0,1
3,3997,0,69,61.450000,0,1,0,1,0,0,...,1,0,0,0,0,1,0,1,0,0
4,6654,0,7,86.500000,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,1476,0,2,20.250000,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,1
10344,1138,0,11,65.150000,0,1,1,1,1,1,...,1,0,0,1,0,0,1,0,0,0
10345,5125,0,11,94.000000,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
10346,2376,0,11,61.250000,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0


# Regressão Logistica

##### A regressão logística é um método popular para prever uma resposta categórica, podemos prever um resultado binário usando regressão logiística binominal ou podemos prever um resultado multiclasse usando regressão logistica multinominal

In [8]:
from pyspark.ml.feature import VectorAssembler

In [10]:
# Renomeando a coluna
dataset = dataset.withColumnRenamed('Churn', 'label')

In [12]:
dataset.limit(1).toPandas()

,id,Mais65anos,MesesDeContrato,MesesCobrados,label,Conjuge,Dependentes,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,...,Internet_DSL,Internet_FibraOptica,Internet_Nao,TipoContrato_Mensalmente,TipoContrato_UmAno,TipoContrato_DoisAnos,MetodoPagamento_DebitoEmConta,MetodoPagamento_CartaoCredito,MetodoPagamento_BoletoEletronico,MetodoPagamento_Boleto
0,7982,0,1,45.305408,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0


In [14]:
# Precisamos criar as features, que são todas as colunas que serão usadas na previsão
# Importante: O 'label' não pode estar na features, pois é o valor que precisamos descobrir
X = dataset.columns
X.remove('label')
X.remove('id')

In [15]:
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [16]:
assembler = VectorAssembler(inputCols=X, outputCol='features')

In [20]:
# No transform passamos o dataset que é usada para previsão, e faremos o select com as variaveis que criamos
dataset_preparado = assembler.transform(dataset).select('features', 'label')

In [23]:
dataset_preparado.show(10, False)

+-----------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                   |label|
+-----------------------------------------------------------------------------------------------------------+-----+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |1    |
|(24,[1,2,3,5,6,8,9,11,12,13,15,17,22],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[1,2,5,6,10,11,12,13,14,18,23],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |0    |
|(24,[1,2,3,5,8,12,13,14,19,21],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |0    |
|(24,[1,2,3,5,6,11,13,15,17,22],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |1    |
|(24,[1,2,5,6,12,13,15,17,22],[14.0,85.03742670311915,1.0,1.0,1.0,1.0,1.

# Ajuste e previsão

#### Para começar o treinamento precisamos separa nossos dados entre "treino" e "teste"

In [26]:
v_seed = 101

In [27]:
# Por padrão são usados 70% para treino e 30% para teste
treino, teste = dataset_preparado.randomSplit([0.7, 0.3], seed=v_seed)

In [31]:
treino.count()

7206

In [32]:
teste.count()

3142

# Usando a Regressão Logística

In [33]:
from pyspark.ml.classification import LogisticRegression

In [34]:
lr = LogisticRegression()

In [35]:
# Criando Modelo para ajustar o modelo de treino
modelo_lr = lr.fit(treino)

In [36]:
# Classificando o modelo de teste
previsoes_lr_teste = modelo_lr.transform(teste)

In [37]:
previsoes_lr_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[3.02174179751551...|[0.95354674000282...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.0922192966076...|[0.47696150091605...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.18744121711361...|[0.54672358463156...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.91716501260103...|[0.71446410549163...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1495904711610...|[0.46267196467801...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1680594619286...|[0.45808374494006...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-1.4170949608173...|[0.19511740608882...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[0.14194260698794...|[0.53542619200881...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.67046644011599...|[0.66160759507905...|       0.0|
|(24,[0,1,2,3,4,